# Importing libraries

In [2]:
# Run if not everything installed
!pip install tensorflow     > null 2>&1
!pip install seaborn        > null 2>&1
!pip install numpy          > null 2>&1
!pip install pillow         > null 2>&1
!pip install opencv-python  > null 2>&1
!pip install scikit-learn   > null 2>&1
#!pip install torchvision    > null 2>&1

In [3]:
#import tensorflow as tf
#import seaborn as sns
import numpy as np

from PIL import Image
import glob
from collections import defaultdict
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers

from sklearn.metrics import mean_squared_error, roc_auc_score, classification_report

# from torchvision.transforms import (
#     Compose,
#     RandomHorizontalFlip,
#     RandomRotation,
# )

# Preprocessing

In [4]:
IMG_SIZE = (94, 125)

In [5]:
# Removing image transparency on pngs to prevent artefacts
def remove_transparency (img):
    if img.mode in ('RGBA', 'LA'):  # Check if the image has an alpha channel
        bg = Image.new("RGB", IMG_SIZE, "black")    # create solid black bg
        bg.paste(img, mask=img.split()[3])      # Paste using alpha channel as a mask
        return bg   # return resulting image
    return img      # if no alpha channel return image unmodified

In [6]:
# Specifying final image size and resizing to that size
def pixels_from_path(file_path):
    im = Image.open(file_path)
    im = im.resize(IMG_SIZE)
    im = remove_transparency(im)
    im = im.convert("RGB")
    np_im = np.array(im)
    # Returns 3D array of RGB values for image
    return np_im

In [7]:
# Making sets of training images
def compile_set(file_path, size):
    new_set = np.asarray([
        pixels_from_path(animal)
        for animal in glob.glob(file_path)[:size]
        if pixels_from_path(animal) is not None
    ])

    return new_set

In [8]:
# Making sets of validation images
def compile_valid_set(file_path, size):
    new_set = np.asarray([
        pixels_from_path(animal)
        for animal in glob.glob(file_path)[-size:]
        if pixels_from_path(animal) is not None
    ])

    return new_set

In [9]:
# Input relevant filepath before '/cat'
cat_filepath = 'cat/*'
dog_filepath = 'dog/*'
tiger_filepath = 'tiger/*'
lion_filepath = 'lion/*'

In [10]:
shape_counts = defaultdict(int)
for i, cat in enumerate(glob.glob(cat_filepath)):
    # Will print iteration no. if i+1 is a multiple of 500
    if (i+1)%500==0:
        print(i)
    img_shape = pixels_from_path(cat).shape
    shape_counts[str(img_shape)]= shape_counts[str(img_shape)]+ 1

499
999
1499
1999
2499
2999
3499
3999
4499
4999
5499
5999
6499
6999
7499
7999
8499
8999
9499
9999
10499
10999
11499
11999
12499


In [11]:
shape_items = list(shape_counts.items())
shape_items.sort(key = lambda x: x[1])
shape_items.reverse()

In [12]:
# 10% of the data will be used for validation
validation_size = 0.1
img_size = IMG_SIZE # resize images to be 0.25x most common shape (374x500)
num_channels = 3 # RGB
sample_size = 25000 # Using all training data for the sample size

In [13]:
len(glob.glob(cat_filepath))

12500

In [14]:
len(glob.glob(lion_filepath))

2621

In [15]:
len(glob.glob(tiger_filepath))

2480

In [16]:
pixels_from_path(glob.glob(cat_filepath)[5]).shape

(125, 94, 3)

In [17]:
# Training size
SAMPLE_SIZE = 11250
SAMPLE_SIZE_TUNING = 2200    # different due to different dataset size

# Validation size
valid_size = 1250
valid_size_tuning = 220      # different due to different dataset size

In [18]:
print("loading training cat images...")
cat_train_set = compile_set(cat_filepath, SAMPLE_SIZE)

print("loading training dog images...")
dog_train_set = compile_set(dog_filepath, SAMPLE_SIZE)

print("loading training tiger images...")
tiger_train_set = compile_set(tiger_filepath, SAMPLE_SIZE_TUNING)

print("loading training lion images...")
lion_train_set = compile_set(lion_filepath, SAMPLE_SIZE_TUNING)

loading training cat images...
loading training dog images...
loading training tiger images...
loading training lion images...


C:\Users\Rachel\IdeaProjects\DLE305-Assessment-2\.venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [19]:
print("loading validation cat images...")
cat_valid_set = compile_valid_set(cat_filepath, valid_size)

print("loading validation dog images...")
dog_valid_set = compile_valid_set(dog_filepath, valid_size)

print("loading training tiger images...")
tiger_valid_set = compile_valid_set(tiger_filepath, valid_size_tuning)

print("loading training lion images...")
lion_valid_set = compile_valid_set(lion_filepath, valid_size_tuning)

loading validation cat images...
loading validation dog images...
loading training tiger images...
loading training lion images...


In [20]:
x_train = np.concatenate([cat_train_set, dog_train_set])
# Applying labels based on sample size because data are currently ordered by class
labels_train = np.asarray([1 for _ in range(SAMPLE_SIZE)]+[0 for _ in range(SAMPLE_SIZE)])

In [21]:
x_tune = np.concatenate([tiger_train_set, lion_train_set])
# Applying labels based on sample size because data are currently ordered by class
labels_tune = np.asarray([1 for _ in range(SAMPLE_SIZE_TUNING)]+[0 for _ in range(SAMPLE_SIZE_TUNING)])

In [22]:
x_valid = np.concatenate([cat_valid_set, dog_valid_set])
# Applying labels based on validation sample size because data are currently ordered by class
labels_valid = np.asarray([1 for _ in range(valid_size)]+[0 for _ in range(valid_size)])

In [23]:
x_valid_tune = np.concatenate([tiger_valid_set, lion_valid_set])
# Applying labels based on validation sample size because data are currently ordered by class
labels_valid_tune = np.asarray([1 for _ in range(valid_size_tuning)]+[0 for _ in range(valid_size_tuning)])

In [24]:
# Reshape labels to match output
labels_train = labels_train.reshape(-1,1)
labels_valid = labels_valid.reshape(-1,1)

In [25]:
labels_tune = labels_tune.reshape(-1,1)
labels_valid_tune = labels_valid_tune.reshape(-1,1)

In [26]:
x_train.shape

(22500, 125, 94, 3)

In [27]:
labels_train.shape

(22500, 1)

In [28]:
labels_train[:10]  # Checking values to ensure they're not None

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [29]:
labels_train[22490:]  # Checking values to ensure they're not None

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [30]:
# Fully connected layer neuron number
fc_layer_size = 256

# CNN from A2

In [31]:
# Convolution parameters
conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

catdog_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [32]:
customAdam = keras.optimizers.Adam(learning_rate=1e-6)
catdog_model.compile(optimizer=customAdam,  # Optimizer
                        # Loss function to minimize
                        loss="BinaryCrossentropy",
                        # List of metrics to monitor
                        metrics=["BinaryCrossentropy","MeanSquaredError", "accuracy"])

In [33]:
print('# Fit model on training data')

history = catdog_model.fit(x_train,
                              labels_train,
                              batch_size=64,
                              shuffle = True,
                              epochs=30,
                              validation_data=(x_valid, labels_valid))

# Fit model on training data
Epoch 1/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 174s 491ms/step - BinaryCrossentropy: 1.7523 - MeanSquaredError: 0.3841 - accuracy: 0.5280 - loss: 1.7523 - val_BinaryCrossentropy: 0.9549 - val_MeanSquaredError: 0.2865 - val_accuracy: 0.6040 - val_loss: 0.9549
Epoch 2/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 171s 486ms/step - BinaryCrossentropy: 0.9133 - MeanSquaredError: 0.2800 - accuracy: 0.6116 - loss: 0.9133 - val_BinaryCrossentropy: 0.8187 - val_MeanSquaredError: 0.2589 - val_accuracy: 0.6324 - val_loss: 0.8187
Epoch 3/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 170s 482ms/step - BinaryCrossentropy: 0.7693 - MeanSquaredError: 0.2490 - accuracy: 0.6392 - loss: 0.7693 - val_BinaryCrossentropy: 0.7608 - val_MeanSquaredError: 0.2460 - val_accuracy: 0.6388 - val_loss: 0.7608
Epoch 4/30
352/352 ━━━━━━━━━━━━━━━━━━━━ 169s 481ms/step - BinaryCrossentropy: 0.6911 - MeanSquaredError: 0.2276 - accuracy: 0.6664 - loss: 0.6911 - val_BinaryCrossentropy: 0.8202 - val_MeanSquaredError: 0.2623 - val

In [34]:
# Evaluating CNN model predictions on validation data

#preds = np.asarray(preds).flatten()
labels_flat = np.asarray(labels_valid).flatten()

preds = catdog_model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])

np.corrcoef(preds, labels_flat)

79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step


array([[1.        , 0.55800595],
       [0.55800595, 1.        ]])

In [35]:
# MSE for predictions
# Closer to 0 is better
print(mean_squared_error(labels_flat, preds))

0.1819567997760077


In [36]:
# Receiver Operating Characteristic and Area Under Curve
# Closer to 1 is better
print(roc_auc_score(labels_flat, preds))

0.82149952


In [37]:
# Saving model
catdog_model.save('untuned_model.keras')

# Fine-tuning model on big cats

In [38]:
# Loading model to variable
untuned_model = keras.models.load_model('untuned_model.keras')

In [39]:
for layer in untuned_model.layers:
    if isinstance(layer, layers.Conv2D):
        layer.trainable = False

In [40]:
# Remove classification head
conv_x = untuned_model.get_layer("flattened_features").output

# Add new dense layers for big cats
conv_x = layers.Dense(fc_layer_size, activation='relu', name='new_fc')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='bigcat_class')(conv_x)

# Create new model
bigcat_model = keras.Model(inputs=untuned_model.input, outputs=conv_outputs)

In [41]:
customAdam = keras.optimizers.Adam(learning_rate=1e-6)  # Small learning rate for transfer learning
bigcat_model.compile(optimizer=customAdam,
                         loss="BinaryCrossentropy",
                         metrics=["BinaryCrossentropy","MeanSquaredError", "accuracy"])

In [42]:
history = bigcat_model.fit(x_tune,
                               labels_tune,
                               batch_size=64,
                               shuffle=True,
                               epochs=15,  # Start with fewer epochs
                               validation_data=(x_valid_tune, labels_valid_tune))

Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 15s 211ms/step - BinaryCrossentropy: 5.3465 - MeanSquaredError: 0.4190 - accuracy: 0.5501 - loss: 5.3465 - val_BinaryCrossentropy: 1.9373 - val_MeanSquaredError: 0.3217 - val_accuracy: 0.6295 - val_loss: 1.9373
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 14s 208ms/step - BinaryCrossentropy: 1.4952 - MeanSquaredError: 0.2658 - accuracy: 0.6829 - loss: 1.4952 - val_BinaryCrossentropy: 1.4965 - val_MeanSquaredError: 0.2723 - val_accuracy: 0.7000 - val_loss: 1.4965
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 14s 207ms/step - BinaryCrossentropy: 1.1286 - MeanSquaredError: 0.2279 - accuracy: 0.7252 - loss: 1.1286 - val_BinaryCrossentropy: 1.2901 - val_MeanSquaredError: 0.2447 - val_accuracy: 0.7068 - val_loss: 1.2901
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 14s 207ms/step - BinaryCrossentropy: 0.9938 - MeanSquaredError: 0.2102 - accuracy: 0.7442 - loss: 0.9938 - val_BinaryCrossentropy: 1.1559 - val_MeanSquaredError: 0.2324 - val_accuracy: 0.7364 - val_loss: 1.1559
Epoc

In [43]:
# Evaluating finetuned model predictions on validation data
labels_flat = np.asarray(labels_valid_tune).flatten()
preds = bigcat_model.predict(x_valid_tune)
preds = np.asarray([pred[0] for pred in preds])

print(np.corrcoef(preds, labels_flat))

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
[[1.         0.53297937]
 [0.53297937 1.        ]]


In [44]:
# MSE for predictions
# Closer to 0 is better
print(mean_squared_error(labels_flat, preds))

0.2157295587663383


In [45]:
# Receiver Operating Characteristic and Area Under Curve
# Closer to 1 is better
print(roc_auc_score(labels_flat, preds))

0.8334504132231404


In [46]:
bigcat_model.save("tuned_model.keras")

Optional for larger datasets

In [47]:
for layer in bigcat_model.layers:
    if isinstance(layer, layers.Conv2D):
        layer.trainable = True  # Unfreeze convolutional layers

In [48]:
customAdam = keras.optimizers.Adam(learning_rate=1e-7)  # Even smaller learning rate
bigcat_model.compile(optimizer=customAdam,
                         loss="BinaryCrossentropy",
                         metrics=["BinaryCrossentropy","MeanSquaredError", "accuracy"])

history_fine = bigcat_model.fit(x_tune,
                                    labels_tune,
                                    batch_size=64,
                                    shuffle=True,
                                    epochs=5,       # Fewer epochs
                                    validation_data=(x_valid_tune, labels_valid_tune))

Epoch 1/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 34s 477ms/step - BinaryCrossentropy: 0.2492 - MeanSquaredError: 0.0628 - accuracy: 0.9227 - loss: 0.2492 - val_BinaryCrossentropy: 0.8614 - val_MeanSquaredError: 0.1926 - val_accuracy: 0.7659 - val_loss: 0.8614
Epoch 2/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 33s 479ms/step - BinaryCrossentropy: 0.2333 - MeanSquaredError: 0.0591 - accuracy: 0.9264 - loss: 0.2333 - val_BinaryCrossentropy: 0.8515 - val_MeanSquaredError: 0.1889 - val_accuracy: 0.7750 - val_loss: 0.8515
Epoch 3/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 33s 477ms/step - BinaryCrossentropy: 0.2125 - MeanSquaredError: 0.0555 - accuracy: 0.9303 - loss: 0.2125 - val_BinaryCrossentropy: 0.8437 - val_MeanSquaredError: 0.1878 - val_accuracy: 0.7773 - val_loss: 0.8437
Epoch 4/5
69/69 ━━━━━━━━━━━━━━━━━━━━ 33s 478ms/step - BinaryCrossentropy: 0.1987 - MeanSquaredError: 0.0547 - accuracy: 0.9304 - loss: 0.1987 - val_BinaryCrossentropy: 0.8395 - val_MeanSquaredError: 0.1874 - val_accuracy: 0.7773 - val_loss: 0.8395
Epoch 5/

In [49]:
# Evaluating finetuned model predictions on validation data
labels_flat = np.asarray(labels_valid_tune).flatten()
preds = bigcat_model.predict(x_valid_tune)
preds = np.asarray([pred[0] for pred in preds])

print(np.corrcoef(preds, labels_flat))

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
[[1.         0.56668521]
 [0.56668521 1.        ]]


In [50]:
# MSE for predictions
# Closer to 0 is better
print(mean_squared_error(labels_flat, preds))

0.19567262130401053


In [51]:
# Receiver Operating Characteristic and Area Under Curve
# Closer to 1 is better
print(roc_auc_score(labels_flat, preds))

0.8433677685950413


In [52]:
bigcat_model.save("retuned_model.keras")